### 순방향 언어 모델
---
우리는 임의의 단어 시퀀스가 해당 언어에서 얼마나 자연스러운지 이해하고 있는 언어 모델을 구축하려고 합니다. 그런데 조건부 확률의 정의에 따라 수식 3-4의 좌변과 우변이 같다는 사실을 알고 있으므로 언어 모델의 학습 방식을 **이전 단어들(컨텍스트)이 주어졌을 때 다음 단어 맞히기**로 정해도 목표를 달성할 수 있습니다.  

다음 그림은 학습 말뭉치가 **어제 카페 갔었어 거기 사람 많더라**라는 문장 하나일 때 언어 모델이 계산하는 대상을 나타낸 것입니다. 검은색 단어는 컨텍스트, 주황색 단어는 맞혀야 할 다음 단어를 의미합니다. 이처럼 문장 앞부터 뒤로, 사람이 이해하는 순서대로 계산하는 모델을 **순방향 언어 모델(forward language model)**이라고 합니다. GPT, ELMo 같은 모델이 이런 방식으로 프리트레인을 수행합니다. 

<center><그림 3-1 순방향 언어 모델></center>  


>**어제**  

>>어제 **카페**  

>>>어제 카페 **갔었어**  

>>>>어제 카페 갔었어 **거기**  

>>>>>어제 카페 갔었어 거기 **사람**  

>>>>>>어제 카페 갔었어 거기 사람 **많더라**


### 역방향 언어 모델
---
다음 그림은 같은 데이터지만 문장 뒤부터 앞으로 계산하는 ***역방향 언어 모델(backward language model)*** 을 나타내고 있습니다. 역방향 언어 모델 역시 방향만 바뀌었을 뿐 다음 단어 맞히기 과정에서 전체 단어 시퀀스가 나타날 확률을 계산할 수 있습니다. ELMo 같은 모델이 이런 방식으로 프리트레인을 수행합니다. 

cf) ELMo(Embeddings from Language Models)는 순방향과 역방향 언어 모델 모두 활용하는 기법입니다. 

<center><그림 3-2 역방향 언어 모델></center>  

>>>>>>**많더라** 

>>>>>**사람** 많더라  

>>>>**거기** 사람 많더라  

>>>**갔었어** 거기 사람 많더라  

>>**카페** 갔었어 거기 사람 많더라  

>**어제** 카페 갔었어 거기 사람 많더라


### 넓은 의미의 언어 모델
---
전통적인 의미의 언어 모델은 수식 3-4 처럼 정의했지만 최근에는 다음 수식처럼 정의하기도 합니다.

<center><수식 3-5 넓은 의미의 언어 모델></center>  
$$P(w|context)$$

이는 컨텍스트(=주변 맥락 정보)가 전제된 상태에서 특정 단어($w$)가 나타날 조건부 확률을 나타냅니다. 이렇게 정의된 언어 모델은 단어나 단어 시퀀스로 구성된 컨텍스트를 입력받아 특정 단어가 나타날 확률을 출력합니다. 이때 컨텍스트와 맞힐 단어를 어떻게 설정하느냐에 따라 다양하게 변형할 수 있습니다.

#### 마스크 언어 모델  

***마스크 언어 모델(masked language model)*** 은 학습 대상 문장에 빈칸을 만들어 놓고 해당 빈칸에 올 단어로 적절한 단어가 무엇일지 분류하는 과정으로 학습합니다. BERT가 마스크 언어 모델로 프리트레인하는 대표적인 모델입니다. 다음 그림은 마스크 언어 모델을 나타낸 것입니다.

<center><그림 3-3 마스크 언어 모델></center>

**어제** 카페 갔었어 거기 사람 많더라  

어제 **카페** 갔었어 거기 사람 많더라  

어제 카페 **갔었어** 거기 사람 많더라  

어제 카페 갔었어 **거기** 사람 많더라  

어제 카페 갔었어 거기 **사람** 많더라  

어제 카페 갔었어 거기 사람 **많더라**  


첫 줄에서 컨텍스트는 **[MASK] 카페 갔었어 거기 사람 많더라** 이고 맞힐 대상 단어는 **어제** 입니다. 마찬가지로 두 번째 줄의 컨텍스트는 **어제** **[MASK]** **갔었어 거기 사람 많더라** 이며 맞힐 단어는 **카페**입니다. 즉, 검정색 단어는 컨텍스트, 주황색 단어는 맞혀야 할 타깃 단어를 가리킵니다.  

맞힐 단어 이전 단어들만 참고할 수 있는 순방향, 역방향 언어 모델과 달리 마스크 언어 모델은 맞힐 단어를 계산할 때 문장 전체의 맥락을 참고할 수 있다는 장점이 있습니다. 이 때문에 마스크 언어 모델에 양방향(bidirectional) 성질이 있다고들 합니다. 다시 말해 맞힐 단어 앞뒤를 모두 본다는 뜻입니다. 


#### 스킵-그램 모델  

***스킵-그램 모델(skip-gram model)*** 은 어떤 단어 앞뒤에 특정 범위를 정해 두고 이 범위 내에 어떤 단어들이 올지 분류하는 과정으로 학습합니다. 다음 그림은 컨텍스트로 설정한 단어(볼드처리한) 앞뒤로 두 개씩 보는 상황을 나타낸 예시입니다.

<center><그림 3-4 스킵-그램 모델></center>

(어제 카페 **갔었어** 거기 사람) 많더라  

어제 (카페 갔었어 **거기** 사람 많더라)  


cf) 책에서는 괄호 대신에 큰 박스로 표시되어 있었지만 편의상 괄호를 사용했습니다.

이때 스킵-그램 모델은 **갔었어** 주변에 **어제, 카페, 거기, 사람**이 나타날 확률을 각각 높이는 방식으로 학습합니다. 그다음에 **거기** 주변에 **카페, 갔었어, 사람, 많더라**가 나타날 확률을 각각 높입니다. 즉, 스킵-그램 모델은 컨텍스트로 설정한 단어 주변에 어떤 단어들이 분포해 있는지를 학습한다는 이야기입니다. 2013년 구글에서 발표한 단어 수준 임베딩 기법인 Word2Vec이 스킵-그램 모델 방식으로 학습합니다.


### 언어 모델의 유용성
---
잘 학습된 언어 모델은 어떤 문장이 자연스러운지 가려낼 수 있어 그 자체로 값어치가 있습니다. 학습 대상 언어의 풍부한 맥락을 포함하고 있다는 점 역시 큰 장점입니다. 이 때문에 기계 번역, 문법 교정, 문장 생성 등 다양한 task를 수행할 수 있습니다. 

- 기계 번역: $P(?|$You can't be free from death)  
- 문법 교정: $P($두시 삼십 이분) > $P($이시 서른 두분)  
- 문장 생성: $P(?|$발 없는 말이)

다음 그림은 Kcbert-large 모델의 계산 결과입니다.  
이 모델은 12GB 크기의 네이버 댓글 데이터로 학습한 BERT인데요, 마스크 언어 모델 방식으로 프리트레인을 했습니다.

https://huggingface.co/beomi/kcbert-large  

cf) 그림을 캡쳐하는 것이 불가능해 해당 링크로 갈음합니다.

이 모델에 **어제 카페 갔었어 [MASK] 사람 많더라**를 입력하니 해당 마스크 위치에 '.(확률값 0.131)'이 오는 것이 가장 자연스럽다고 예측하고 있군요. '?(0.119)', '**생각보다**(0.106)', '~(0.092)', '**어쩐지**(0.054)' 등이 그 뒤를 잇고 있습니다. 모두 그럴듯한 한국어 문장임을 확인할 수 이습니다.  


다음 그림은 OpenAI가 2020년에 공개한 대규모 언어 모델인 GPT3의 단순 계산 능력을 평가한 그래프입니다. '다음 단어 맞히기'라는 단순 태스크로만 프리트레인을 했음에도 가장 큰 모델인 175B는 두 자릿수 덧셈/뺄셈에서 거의 100%에 가까운 정확도를 나타내고 있습니다.  
해당 모델이 학습 말뭉치를 그대로 외운 것 같은 인상을 주지만, 큰 언어 모델에 학습 대상 언어의 풍부한 맥락이 포함됐다는 점은 의심할 여지가 없습니다.  



cf) 모델의 파라미터(parameter) 수가 175Billion, 즉 1,750억 개라는 뜻입니다.  
모델 파라미터는 행렬이나 벡터인데요, 파라미터 수는 행렬, 벡터 요소 수의 총합을 가리킵니다. 파라미터 수가 많을수록 큰 모델이라는 뜻입니다.

https://arxiv.org/abs/2005.14165

cf) 해당 paper 링크로 갈음하겠습니다.

최근 들어 언어 모델이 주목받는 이유 가운데 하나는 데이터 제작 비용 때문입니다. '다음 단어 맞히기'나 '빈칸 맞히기' 등으로 학습 task를 구성하면 사람이 일일이 수작업해야 하는 레이블 없이도 많은 학습 데이터를 싼값에 만들어 낼 수 있습니다. GPT3 같은 대규모 언어 모델이 탄생하게 된 배경이기도 하죠.

또 다른 이유는 'Transfer Learnig'을 꼽을 수 있습니다. 대량의 말뭉치로 프리트레인한 언어 모델을 문서 분류, 개체명 인식 등 다운스트림 task에 적용하면 적은 양의 데이터로도 그 성능을 큰 폭으로 올릴 수 있습니다. 실제로 최근에 제안되는 기법들은 프리트레인을 마친 딥러닝 계얼 언어 모델을 바탕으로 할 때가 많습니다. 이 언어 모델의 최종 또는 중간 출력값을 가지고 다양한 task를 수행합니다.  

cf) 이러한 출력값을 임베딩(Embedding) 또는 리프레젠테이션(Representation)이라고 합니다.